# Classification using an Artificial Neural Network

A simple ANN built using data from a course on Udemy called [Deep Learning A-Z](https://www.udemy.com/course/deeplearning/learn/lecture/20258078#overview). The dataset lists customers at a bank who have either stayed or chosen to leave, along with several data points for each customer such as their bank balance, credit score and geographic location. The aim of this model is to predict whether a given customer will stay or leave the bank, based on the aforementioned data points.